In [3]:
import pandas as pd
import numpy as np
import os
os.chdir(r"C:\Users\dai\Desktop\S-AI\MachineLearning\Day12\Datasets")
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.api import Holt
from statsmodels.tsa.api import SimpleExpSmoothing
from scipy.cluster.hierarchy import linkage,dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.cluster import DBSCAN

In [5]:
nut = pd.read_csv('nutrient.csv',index_col=0)

In [7]:
scaler = StandardScaler().set_output(transform='pandas')
nut_scaled = scaler.fit_transform(nut)

In [9]:
eps_range = [0.2,0.4,0.6,1]
mp_range = [2,3,4,5]
cnt = 0
a =[]
for i in eps_range:
    for j in mp_range:
        clust_DB = DBSCAN(eps=i, min_samples=j)
        clust_DB.fit(nut_scaled.iloc[:,:5])
        if len(set(clust_DB.labels_)) > 2: ## length of the set should be greater than 2 for silhoutte score 
            cnt = cnt + 1
            nut_scaled['clust'] = clust_DB.labels_
            nut_inliers = nut_scaled[nut_scaled['clust']!=-1]## filtering the outliers
            sil_sc = silhouette_score(nut_inliers.iloc[:,:-1],# dataframe # last column not consider
                             nut_inliers.iloc[:,-1])# label # the last column
            a.append([cnt,i,j,sil_sc])
            print(i,j,sil_sc)
    
a = np.array(a)
pa = pd.DataFrame(a,columns=['Sr','eps','min_pt','sil'])
print("Best Paramters:")
pa[pa['sil'] == pa['sil'].max()]


0.2 2 0.5710837307456915
0.4 2 0.9040570858308878
0.6 2 0.7752441663222646
1 2 0.43236957939711584
1 3 0.43147150560271585
Best Paramters:


,Sr,eps,min_pt,sil
1,2.0,0.4,2.0,0.904057


In [11]:
clust_DB = DBSCAN(eps = 0.4 , min_samples=2)
clust_DB.fit(nut_scaled)


DBSCAN(eps=0.4, min_samples=2)

In [13]:
nut_clust = nut.copy()
nut_clust['cluster'] = clust_DB.labels_
nut_clust  = nut_clust[nut_clust['cluster'] != -1]
nut_clust.groupby('cluster').mean()

,energy,protein,fat,calcium,iron
cluster,,,,,
0,350.0,19.4,29.4,9.0,2.52
1,170.0,25.0,7.0,9.5,1.35


In [39]:
nut_clust.sort_values

<bound method DataFrame.sort_values of                 energy  protein  fat  calcium  iron  cluster
Food_Item                                                   
BEEF BRAISED       340       20   28        9   2.6        0
BEEF STEAK         375       19   32        9   2.6        0
CHICKEN CANNED     170       25    7       12   1.5        1
SMOKED HAM         340       20   28        9   2.5        0
PORK ROAST         340       19   29        9   2.5        0
PORK SIMMERED      355       19   30        9   2.4        0
TUNA CANNED        170       25    7        7   1.2        1>

In [15]:
clust_DB.labels_

array([ 0, -1, -1,  0, -1, -1,  1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  1, -1], dtype=int64)

In [31]:
nut_inliers

,energy,protein,fat,calcium,iron,clust
Food_Item,,,,,,
BEEF BRAISED,1.335059,0.239681,1.314297,-0.456581,0.152385,0
HAMBURGER,0.378515,0.479361,0.318516,-0.456581,0.222121,0
BEEF STEAK,1.687470,0.000000,1.676399,-0.456581,0.152385,0
BEEF CANNED,-0.275962,0.719042,-0.315163,-0.352109,0.919476,1
CHICKEN BROILED,-0.930439,0.239681,-0.948842,-0.469640,-0.684442,1
CHICKEN CANNED,-0.376651,1.438084,-0.586740,-0.417404,-0.614706,1
LAMB LEG ROAST,0.579893,0.239681,0.590093,-0.456581,0.152385,0
LAMB SHOULDER ROAST,0.932304,-0.239681,1.042720,-0.456581,-0.056822,0
SMOKED HAM,1.335059,0.239681,1.314297,-0.456581,0.082650,0
